<h1 style="text-align: center; vertical-align: middle;">Numerical Methods in Accelerator Physics</h1>
<h2 style="text-align: center; vertical-align: middle;">Python examples -- Week 3</h2>

<h2>Run this first!</h2>

Imports and modules:

In [ ]:
from config3 import *
%matplotlib inline
import PyNAFF

<h3>Pendulum parameters, Hamiltonian</h3>

In [ ]:
m = 1 # point mass
g = 1 # magnitude of the gravitational field
L = 1 # length of the rod

In [ ]:
### Values of hamiltonian for comparison with theory
TH, PP = np.meshgrid(np.linspace(-np.pi * 1.1, np.pi * 1.1, 100), 
                     np.linspace(-3, 3, 100))

HH = hamiltonian(TH, PP)

<h2>Statistics</h2>

<h3>Collection of 105 pendulums</h3>

In [ ]:
### Initializing the pendulums on a grid
theta_grid = np.linspace(-0.5 * np.pi, 0.5 * np.pi, 21)
p_grid = np.linspace(-0.3, 0.3, 5)

thetas, ps = np.meshgrid(theta_grid, p_grid)

In [ ]:
### Number of pendulums
N = len(theta_grid) * len(p_grid)
N

In [ ]:
### Initial phase space
plt.scatter(thetas, ps, c='b', marker='.')

plot_hamiltonian()

<h3>Long-term behaviour (3000 tracking steps)</h3>

In [ ]:
### Number of time steps
n_steps = 3000

In [ ]:
### Tracking using Leapfrog and adding a constant offset in theta at the beginning
results_thetas3 = np.zeros((n_steps, N), dtype=np.float32)
results_thetas3[0] = thetas.flatten() + 0.1 * np.pi

results_ps3 = np.zeros((n_steps, N), dtype=np.float32)
results_ps3[0] = ps.flatten()

for k in range(1, n_steps):
    results_thetas3[k], results_ps3[k] = solve_leapfrog(results_thetas3[k - 1], results_ps3[k - 1])

In [ ]:
### Plot centroid amplitude, variance and emittance
plot_macro_evolution(results_thetas3, results_ps3);

<p>Initial centroid offset decreases but variance and emittance increase at the same time (filamentation!).</p>

<h2>Oscillation frequencies</h2>

<h3>Employ FFT on pendulum motion (slide 21)</h3>

In [ ]:
## Observe two particles:
i1 = N // 2 - 10
i2 = N // 2

plt.plot(results_thetas3[:, i1])
plt.plot(results_thetas3[:, i2])
plt.xlabel('Steps $k$')
plt.ylabel(r'$\theta$');

In [ ]:
## Obtain the FFT spectra for both:
spec1 = np.fft.rfft(results_thetas3[:, i1])
spec2 = np.fft.rfft(results_thetas3[:, i2])

freq = np.fft.rfftfreq(n_steps)

## Frequency of linearized system
freq_theory = np.sqrt(1 / 1) * 0.1 / (2 * np.pi)

In [ ]:
plt.plot(freq, np.abs(spec1))
plt.plot(freq, np.abs(spec2))
plt.axvline(freq_theory, c='k', zorder=0, ls='--')
plt.xlim(0, 0.05)
plt.xlabel('Phase advance per $\Delta t$ [$2\pi$]')
plt.ylabel('FFT spectrum')
plt.yscale('log');

<h3>Frequency vs Amplitude</h3>

In [ ]:
specs = np.abs(np.fft.rfft(results_thetas3.T))

In [ ]:
max_ids = np.argmax(specs, axis=1)
amplitudes = np.max(specs, axis=1)

In [ ]:
i_range = np.where(ps.flatten() == 0)[0]

plt.scatter(results_thetas3[0][i_range], freq[max_ids][i_range])

plt.axhline(freq_theory, c='k', ls='--', zorder=0)
plt.xticks([-np.pi/2, 0, np.pi/2, np.pi], [r"$-\pi/2$", "0", r"$\pi/2$", r"$\pi$"])
plt.xlabel(r'Initial $\theta$ ($p=0$)')
plt.ylabel('Phase advance / \n' + r'integration step $\Delta t$ [$2\pi$]');
plt.ylim(0.012, 0.0165);

<h3>NAFF Algorithm (slide 22)</h3>

In [ ]:
freqs_naff = []

for signal in results_thetas3.T[i_range]:
    freq_naff = PyNAFF.naff(signal, turns=n_steps - 1, nterms=1)
    try:
        freq_naff = freq_naff[0, 1]
    except IndexError:
        freq_naff = 0
    freqs_naff += [freq_naff]

freqs_naff = np.array(freqs_naff)

In [ ]:
plt.scatter(results_thetas3[0][i_range], freq[max_ids][i_range])
plt.plot(results_thetas3[0][i_range], freqs_naff, c='r', marker='.')

plt.axhline(freq_theory, c='k', ls='--', zorder=0)
plt.xticks([-np.pi/2, 0, np.pi/2, np.pi], [r"$-\pi/2$", "0", r"$\pi/2$", r"$\pi$"])
plt.xlabel(r'Initial $\theta$ ($p=0$)')
plt.ylabel('Phase advance / \n' + r'integration step $\Delta t$ [$2\pi$]');

In [ ]:
plt.plot(results_thetas3[0][i_range], 100 * (freq[max_ids][i_range] - freqs_naff) / freqs_naff)
plt.xlabel(r'Initial $\theta$ ($p=0$)')
plt.ylabel('Error (FFT - NAFF) / NAFF ' + r'[%]');

<h2>Deterministic chaos</h2>

<h3>Discrete pendulum (slide 32)</h3>

In [ ]:
N = 11

thetas = np.linspace(0, 0.99 * np.pi, 11)
ps = np.zeros_like(thetas)

In [ ]:
plt.scatter(thetas, ps, c='b', marker='.')

plot_hamiltonian()

<h3>Time evolution (using Leapfrog)</h3>

In [ ]:
n_steps = 1000

In [ ]:
results_thetas = np.zeros((n_steps, N), dtype=np.float32)
results_thetas[0] = thetas

results_ps = np.zeros((n_steps, N), dtype=np.float32)
results_ps[0] = ps

for k in range(1, n_steps):
    results_thetas[k], results_ps[k] = solve_leapfrog(results_thetas[k - 1], results_ps[k - 1])

In [ ]:
plt.scatter(results_thetas, results_ps, c='b', marker='.', s=1)

plot_hamiltonian()

<h3>Chaos near unstable fixed point</h3>

Let us investigate the unbounded, continuously rotating pendulum, with an energy just above the separatrix value:

In [ ]:
N = 2

thetas = np.pi * np.ones(N, dtype=np.float32)
ps = np.linspace(0.01, 0.05, N)

In [ ]:
results_thetas2 = np.zeros((n_steps, N), dtype=np.float32)
results_thetas2[0] = thetas

results_ps2 = np.zeros((n_steps, N), dtype=np.float32)
results_ps2[0] = ps

for k in range(1, n_steps):
    results_thetas2[k], results_ps2[k] = solve_leapfrog(results_thetas2[k - 1], results_ps2[k - 1])

In [ ]:
plt.scatter(results_thetas2 % (2 * np.pi), results_ps2, c='b', marker='.', s=1)
plt.scatter([np.pi], [0], c='r', marker='o')

plt.xlim(np.pi - 0.1, np.pi + 0.1)
plt.ylim(-0.01, 0.1)
plt.xlabel(r'$\theta$')
plt.ylabel(r'$p$');

<h3>Qualitative Investigation of Local Lyapunov Exponent</h3>

First investigate around stable fixed point $\theta=0$, $p=0$.

In [ ]:
dist = 1e-10

thetas = 0 * np.pi * np.ones(N, dtype=np.float64) # change this line
ps = np.array([0.001, 0.001 + dist], dtype=np.float64)

In [ ]:
n_steps = 100000

In [ ]:
results_thetas3 = np.zeros((n_steps, N), dtype=np.float64)
results_thetas3[0] = thetas

results_ps3 = np.zeros((n_steps, N), dtype=np.float64)
results_ps3[0] = ps

for k in range(1, n_steps):
    results_thetas3[k], results_ps3[k] = solve_leapfrog(results_thetas3[k - 1], results_ps3[k - 1])

In [ ]:
plt.plot(results_thetas3[:, 0], label='$p_{ini}$')
plt.plot(results_thetas3[:, 1], label='$p_{ini} + \Delta p_{ini}$')

#plt.xlim(100000-1000, 100000)

plt.xlabel('Steps $k$')
plt.ylabel(r'$\theta$')
plt.legend();

In [ ]:
plt.plot(results_thetas3[:, 0], label='$p_{ini}$')
plt.plot(results_thetas3[:, 1], label='$p_{ini} + \Delta p_{ini}$')

plt.xlim(100000-1000, 100000)

plt.xlabel('Steps $k$')
plt.ylabel(r'$\theta$')
plt.legend();

In [ ]:
results_dist = np.sqrt(
    (results_thetas3[:, 1] - results_thetas3[:, 0])**2 + 
    (results_ps3[:, 1] - results_ps3[:, 0])**2
)

In [ ]:
plt.plot(results_dist)

#plt.xlim(100000-1000, 100000)

plt.yscale('log')
plt.xlabel('Steps $k$')
plt.ylabel('Phase-space distance');

In [ ]:
plt.plot(results_dist)

plt.xlim(100000-1000, 100000)

plt.yscale('log')
plt.xlabel('Steps $k$')
plt.ylabel('Phase-space distance');

Let us now investigate around unstable fixed point $\theta=\pi$, $p=0$.

In [ ]:
thetas2 = np.pi * np.ones(N, dtype=np.float64) # change this line

In [ ]:
results_thetas4 = np.zeros((n_steps, N), dtype=np.float64)
results_thetas4[0] = thetas2

results_ps4 = np.zeros((n_steps, N), dtype=np.float64)
results_ps4[0] = ps

for k in range(1, n_steps):
    results_thetas4[k], results_ps4[k] = solve_leapfrog(results_thetas4[k - 1], results_ps4[k - 1])

In [ ]:
plt.plot(results_thetas4[:, 0], label='$p_{ini}$')
plt.plot(results_thetas4[:, 1], label='$p_{ini} + \Delta p_{ini}$')

#plt.xlim(100000-1000, 100000)

plt.xlabel('Steps $k$')
plt.ylabel(r'$\theta$')
plt.legend();

In [ ]:
results_dist2 = np.sqrt(
    (results_thetas4[:, 1] - results_thetas4[:, 0])**2 + 
    (results_ps4[:, 1] - results_ps4[:, 0])**2
)

In [ ]:
plt.plot(results_dist2)

#plt.xlim(100000-1000, 100000)

plt.yscale('log')
plt.xlabel('Steps $k$')
plt.ylabel('Phase-space distance');

<h3>Evaluating local maximum Lyapunov exponent</h3>

Nearly exponential increase over two periods (first 4000 steps), then bounded by system size.

Local Maximum Lyapunov exponent estimated by simple linear regression:

$$\lambda_1 \approx \mathrm{slope}\left(\frac{1}{k\Delta t} \ln\left(\frac{|(\theta_2,p_2)-(\theta_1,p_1)|}{10^{-10}}\right)\right)$$

Near stable fixed point:

In [ ]:
B, A = np.polyfit(
    x=np.arange(n_steps)[:4000],
    y=np.log(results_dist[:4000] / dist),
    deg=1
)

B

In [ ]:
plt.plot(results_dist[:4000] / dist)
plt.plot(np.exp(B * np.arange(n_steps)[:4000] + A))

plt.yscale('log');

Near unstable fixed point:

In [ ]:
B, A = np.polyfit(
    x=np.arange(n_steps)[:4000],
    y=np.log(results_dist2[:4000] / dist),
    deg=1
)

B

In [ ]:
plt.plot(results_dist2[:4000] / dist)
plt.plot(np.exp(B * np.arange(n_steps)[:4000] + A))

plt.yscale('log');

<h3>Frequency diffusion (discrete pendulum - slide 33)</h3>

In [ ]:
n_steps = 100000

In [ ]:
theta_ini = np.pi - 0.001
p_ini = 0

In [ ]:
results_thetas4 = np.zeros(n_steps, dtype=np.float64)
results_thetas4[0] = theta_ini

results_ps4 = np.zeros(n_steps, dtype=np.float64)
results_ps4[0] = p_ini

for k in range(1, n_steps):
    results_thetas4[k], results_ps4[k] = solve_leapfrog(results_thetas4[k - 1], results_ps4[k - 1])

In [ ]:
plt.plot(results_thetas4)

plt.xlim(0, 1000)

plt.xlabel('Steps $k$')
plt.ylabel(r'$\theta$');

In [ ]:
window_length = 1000

freqs_naff = []

for signal in results_thetas4.reshape((n_steps // window_length, window_length)):
    freq_naff = PyNAFF.naff(signal - np.mean(signal), turns=window_length, nterms=1)[0, 1]
    freqs_naff += [freq_naff]

In [ ]:
plt.plot(np.arange(0, n_steps, window_length), freqs_naff, ls='none', marker='.')

plt.xlabel('Steps $k$')
plt.ylabel('NAFF determined frequency');

In [ ]:
plt.figure(figsize=(10, 5))
plt.hist(freqs_naff);